In [3]:
## Importing libraries
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [11]:
## Data Cleaning
# Get energy dummy
sample_data = pd.read_excel('../EXAM/data/cleaned_data_excel.xlsx')
df = pd.DataFrame(sample_data)
df['energy'] = df['energy'].str.replace('Energimærke ', '')
df['energy'] = df['energy'].str.replace('A2015','A')
df['energy'] = df['energy'].str.replace('A2020','A')
dummy_energy = pd.get_dummies(df['energy'])
df = pd.concat([df,dummy_energy],axis=1)

# Change constructing year to construction age and squared term for OLS.
df['age'] = 2023 - df['year']
df['age_squared'] = df['age'] ** 2

df
# df[['price','area_code']].groupby('area_code').mean()




,price,address,city,type,saledays,energy,living_space,ground_space,rooms,owner_expenses,...,A,B,C,D,E,F,G,Villa/Fritidsbolig,age,age_squared
0,1275000,Strandparken 46,Vest- og Sydsjælland,Villa/Fritidsbolig,202,D,170,1019,2,2598,...,0,0,0,1,0,0,0,0,45,2025
1,14750000,Niels Andersens Vej 56,Københavns omegn,Villa,255,E,248,984,8,8643,...,0,0,0,0,1,0,0,0,82,6724
2,2850000,Nøddevænget 20,Sydjylland,Villa,0,C,163,858,5,3343,...,0,0,1,0,0,0,0,0,58,3364
3,3695000,Askeengen 112,Københavns omegn,Villa,0,D,130,649,4,4974,...,0,0,0,1,0,0,0,0,53,2809
4,2095000,Rismøllegade 32,Østjylland,Villa,0,D,173,1335,7,2673,...,0,0,0,1,0,0,0,0,54,2916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,1750000,"Prins Haralds Allé 101, 1.",Fyn,Villa,48,G,109,506,3,1922,...,0,0,0,0,0,0,1,0,88,7744
3002,990000,Nissumvej 4,Vestjylland,Villa/Fritidsbolig,48,C,117,800,3,1566,...,0,0,1,0,0,0,0,0,81,6561
3003,1695000,Baunetoften 12,Nordsjælland,Rækkehus,240,C,86,103,4,2184,...,0,0,1,0,0,0,0,0,33,1089
3004,650000,Aalevej 40,Østjylland,Villa,48,D,79,829,3,1464,...,0,0,0,1,0,0,0,0,68,4624


In [ ]:
## Dataset split
columns_to_drop = ['price','address','city','type','energy', 'year','area_name', '']
X,y = df.drop(columns_to_drop,axis=1),df.price
test_size = 0.30 # taking 70:30 training and test set
seed = 17082023 # Random number seeding for repeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [ ]:
## Main model
eta = 0.1
gradient_booster = HistGradientBoostingClassifier(learning_rate=eta)
# Gradient boosting classifiers are required to implement gradient boosting.
gradient_booster.fit(X_train,y_train)
y_test_predict = gradient_booster.predict(X_test)
print(classification_report(y_test,y_test_predict))
# Use the Python module named classification report to verify the correctness and quality of the accuracy report ().

In [ ]:
## Lasso regularization
# Normal scaler
from sklearn.preprocessing import StandardScaler
norm_scaler = StandardScaler().fit(X_train)
X_train_std = norm_scaler.transform(X_train)
X_test_std = norm_scaler.transform(X_test)

# Lasso
from sklearn.linear_model import Lasso
output = []
lambdas = np.logspace(-4, 4, 20)
for lambda_ in lambdas:
    reg = Lasso(alpha=lambda_, random_state=1)
    reg.fit(X_train_std, y_train)
    output.append(
        [
            lambda_,
            mse(reg.predict(X_train_std), y_train,squared=False),
            mse(reg.predict(X_test_std), y_test,squared=False),
        ]
    )

In [ ]:
## Search for best lambda
MSE_df = pd.DataFrame(
    data=output, columns=["lambda", "MSE train", "MSE test"]
).set_index("lambda")

MSE_df.plot(logx=True, logy=True)

# find the minimal observations as a series
best_fit = MSE_df["MSE test"].nsmallest(1)

# take out the data minimum RMSE and the optimal lambda
lambda_opt, RMSE_min = next(best_fit.items())
print(f"Minimum RMSE = {RMSE_min:.3f} found for lambda = {lambda_opt:.4f}.")


In [ ]:
## 5-fold cross validation
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold
# Pipeline
pipe_lasso = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False),
    StandardScaler(),
    Lasso(random_state=1),
)

# Split X, y into development (2/3) and test data (1/3).
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=(1 / 3), random_state=1)

lambdas = np.logspace(-4, 4, 12)

kfolds = KFold(n_splits=5)
mses = []

for lambda_ in lambdas:

    pipe_lasso = make_pipeline(
        PolynomialFeatures(degree=3, include_bias=False),
        StandardScaler(),
        Lasso(alpha=lambda_, random_state=1),
    )
    mses_test = []
    mses_train = []

    for train_idx, val_idx in kfolds.split(X_dev, y_dev):
        X_train, y_train = X_dev.iloc[train_idx], y_dev[train_idx]
        X_val, y_val = X_dev.iloc[val_idx], y_dev[val_idx]

        pipe_lasso.fit(X_train, y_train)

        mses_train.append(mse(pipe_lasso.predict(X_train), y_train))
        mses_test.append(mse(pipe_lasso.predict(X_val), y_val))

    mses.append([np.mean(mses_train), np.mean(mses_test), lambda_])

# Create df with MSE values
df_mses = pd.DataFrame(mses, columns=["MSE_train", "MSE_test", "lambda"])

# Index of the lambda that gives the lowest MSE_test in the dataframe
idx_optimal_lambda = df_mses.idxmin()["MSE_test"]
lambda_opt_test = df_mses.loc[idx_optimal_lambda]["lambda"]
opt_test_mse = df_mses.loc[idx_optimal_lambda]["MSE_test"]
print(
    f"Lowest test MSE equal to {opt_test_mse:.4f} is"
    f" achieved with lambda = {lambda_opt_test:.3f}."
)

In [ ]:
## Root mean squared errors
# rmse_GBM = mse(y_test,y_test_predict,squared=False)
# rmse_GBM

In [ ]:
## R-squared
r2_GBM = r2_score(y_test,y_test_predict)
r2_GBM